In [1]:
import os
import sys


parent_dir = os.path.abspath(os.path.join(os.path.dirname('../NEUROEVOLUTION'), '..'))
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)
    


parent_dir = os.path.abspath(os.path.join(os.path.dirname('../NEUROEVOLUTION/gpolnel'), '..'))
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

from NEUROEVOLUTION.utils import seed
import csv
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim

from NEUROEVOLUTION.utils import kfold,seed, drop_features, save_best_params

import pickle
import random
random.seed(seed)
import torch
import pickle
torch.manual_seed(seed)

from nn_utils import nn_cross_validation, create_random_model, random_optimizer
target = 'fat'

ModuleNotFoundError: No module named 'gpolnel'

In [ ]:
X = pd.read_csv('../datamart/data_project_nel.csv')
X = drop_features(X)
y = pd.read_csv('../datamart/y_' + f'{target}'+ '.csv')['lactose_percent']

In [ ]:
batch_sizes = [5,10,15,20,25]
learning_rates = [0.0001, 0.001, 0.01, 0.1]
n_epoches = [25, 50, 75, 100, 250, 500, 1000]
weight_decays = [0.0, 0.1, 0.01, 0.001, 0.0001] 
loss_fn = nn.MSELoss()

best_score = np.inf

name = 'logs/' +f'{target}' + '/random_search_total' + '.csv'
with open(name, 'w', newline='\n') as csvfile:
    w = csv.writer(csvfile, delimiter=';')
    w.writerow(['id']+['model']+['optimizer']+['learning_rate']+['l2']+['batch_size']+['n_epochs']+['cv_score'])


for i in range(1,100):

    batch_size = random.choice(batch_sizes)
    model = create_random_model()
    lr = random.choice(learning_rates)
    optimizer_name = random_optimizer()
    n_epochs = random.choice(n_epoches)
    weight_decay = random.choice(weight_decays)
    
    log_path_cv = 'logs/' + f'{target}'+'/' + f'{i}' + '_cv_results.csv'
    log_path_train = 'logs/'+ f'{target}'+ '/train_logs.csv'
    
    score = nn_cross_validation(X_train=X, y_train=y, model=model, loss_fn=loss_fn, optimizer_name = optimizer_name,lr=lr, weight_decay=weight_decay,  n_epochs=n_epochs, batch_size=batch_size,kf=kfold, cv_log_path=log_path_cv, train_log_path=log_path_train)
    print('cv_score:', score)
    with open(name, 'a', newline='\n') as csvfile:
        w = csv.writer(csvfile, delimiter=';')
        w.writerow([i]+[f'{model}']+[f'{optimizer_name}']+[lr]+[weight_decay]+[batch_size]+[n_epochs]+[score])
    
    if score < best_score:
        best_score = score
        print(f'New best cv_score: {best_score} at search {i}')
        best_params = {
            'X': X,
            'y': y,
            'batch_size': batch_size,
            'kfold': kfold,
            'n_epochs': n_epochs,
            'model': model,
            'lr': lr,
            'optimizer_name': optimizer_name,
            'weight_decay': weight_decay,
            'loss_fn': loss_fn,
            'id': i,
        }
        save_best_params('best_params/'+ f'{target}' + '-best_params-' + f'{i}' +'.pkl', best_params)
        
        
save_best_params('best_params/'+ f'{target}' + '-best_params_final.pkl', best_params)

cv_score: 0.0590872
New best cv_score: 0.05908719822764397 at search 1
cv_score: 4.2945633
cv_score: 4.5531583
cv_score: 3.4659553
cv_score: 0.058362048
New best cv_score: 0.058362048119306564 at search 5


KeyboardInterrupt: 